In [30]:
import boto3
import os
import glob
import shutil

from stac_validator import stac_validator

In [2]:
AWS_ACCESS_KEY_ID = "[CHANGE ME]"
AWS_SECRET_ACCESS_KEY = "[CHANGE ME]"
AWS_SESSION_TOKEN = "[CHANGE ME]"

In [9]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
)

In [34]:
# Define the directory path
local_dir = "./collections-to-validate"

In [68]:
def download_collections(bucket_name, s3_prefixes):
    s3 = s3_client
    s3_responses = []
    json_keys = []

    for s3_prefix in s3_prefixes:
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=s3_prefix)
        s3_responses.append(response)

    for item in s3_responses:
        for item in item["Contents"]:
            if item["Key"].endswith(".json"):
                json_keys.append(item["Key"])

    # Create the directory if it doesn't exist
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
        print("Directory created successfully at", local_dir)
    else:
        print("Directory already exists at", local_dir)

    # Download json files to local directory
    for key in json_keys:
        local_file_path = os.path.join(local_dir, os.path.basename(key))
        s3.download_file(bucket_name, key, local_file_path)


def validate_collections():
    json_files = glob.glob(os.path.join(local_dir, "*.json"))
    for json_file in json_files:
        print(f"Validating file: {json_file}")
        stac = stac_validator.StacValidate(json_file)
        stac.run()
        valid_stac = stac.message[0]["valid_stac"]
        print(valid_stac)
        if not valid_stac:
            print(f"Invalid STAC. Error message {stac.message[0]['error_message']}")


def cleanup():
    shutil.rmtree(local_dir)

In [69]:
bucket_name = "veda-data-store"
s3_prefixes = ["hlsl30-ej-reprocessed", "hlss30-ej-reprocessed"]
download_collections(bucket_name, s3_prefixes)

Directory created successfully at ./collections-to-validate


In [70]:
validate_collections()

Validating file: ./collections-to-validate/HLS.L30.T19QHA.2017221T144403.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.S30.T15RYP.2021250T163901.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.S30.T15RYP.2021210T163901.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.S30.T15RYP.2021200T163901.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.L30.T19QHA.2017173T144347.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.S30.T15RYP.2021220T163901.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.S30.T19QHA.2017293T150709.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.L30.T19QHA.2017205T144356.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collections-to-validate/HLS.S30.T15RYP.2021290T164311.v2.0_stac-ej-reprocessed.json
True
Validating file: ./collectio

In [71]:
cleanup()